Table: Matches

* Match_ID (int): unique ID per match
* Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
* Season (int): Season the match took place in (usually covering the period of August till May of the following year)
* Date (str): Date of the match
* HomeTeam (str): Name of the home team
* AwayTeam (str): Name of the away team
* FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
* FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
* FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

Table: Teams

* Season (str): Football season for which the data is valid
* TeamName (str): Name of the team the data concerns
* KaderHome (str): Number of Players in the squad
* AvgAgeHome (str): Average age of players
* ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team
* OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)
* AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)
* StadiumCapacity (str): Maximum stadium capacity of the team's home stadium

Table: Unique Teams

* TeamName (str): Name of a team
* Unique_Team_ID (int): Unique identifier for each team

Table: Teams_in_Matches

* Match_ID (int): Unique match ID
* Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)

In [67]:
# Import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests
import json
import requests

In [69]:
# OOP
class SQL:
    def __init__(self, database):
        self.database = database
        conn = sqlite3.connect(database)
        self.cur = conn.cursor()
    
    def read_sql(self, query):
        df = pd.DataFrame(self.cur.execute(query).fetchall())
        df.columns = [x[0] for x in self.cur.description]
        self.df = df
        return df
pass

In [77]:
database = 'database.sqlite'
df_setup = SQL(database)

In [128]:
query = '''SELECT * 
            from Matches 
            WHERE Season = 2011'''

df_query = df_setup.read_sql(query)
df_query.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [127]:
home_query = '''SELECT HomeTeam,
            COUNT (HomeTeam) * 2 AS Games_Played,
            SUM(FTHG) AS Home_Goals, 
            COUNT(CASE WHEN FTR = 'H' THEN 1 ELSE NULL END) AS Home_Wins,
            COUNT(CASE WHEN FTR = 'D' THEN 1 ELSE NULL END) AS Home_Draw
            FROM Matches
            WHERE Season = '2011'
            GROUP BY 1'''

home_query = df_setup.read_sql(home_query)
home_query.head()

,HomeTeam,Games_Played,Home_Goals,Home_Wins,Home_Draw
0,Aachen,34,15,4,6
1,Arsenal,38,39,12,4
2,Aston Villa,38,20,4,7
3,Augsburg,34,20,6,7
4,Bayern Munich,34,49,14,1


In [92]:
away_query = '''SELECT AwayTeam,
        SUM(FTAG) AS Away_Goals,
        COUNT(CASE WHEN FTR = 'A' THEN 1 ELSE NULL END) AS Away_Wins,
        COUNT(CASE WHEN FTR = 'D' THEN 1 ELSE NULL END) AS Away_Draw
        FROM Matches
        WHERE Season = '2011'
        GROUP BY 1'''

away_query = df_setup.read_sql(away_query)
away_query.head()

,AwayTeam,Away_Goals,Away_Wins,Away_Draw
0,Aachen,15,2,7
1,Arsenal,35,9,3
2,Aston Villa,17,3,10
3,Augsburg,16,2,7
4,Bayern Munich,28,9,3


In [130]:
combined_query = '''
            SELECT h.HomeTeam AS Team,
            h.Games_Played AS Games_Played,
            SUM(h.Home_Goals + a.Away_Goals) AS Total_Goals,
            SUM(h.Home_Wins + a.Away_Wins) AS Total_Wins,
            SUM(h.Home_Draws + a.Away_Draws) AS Total_Draws,
            SUM(h.Games_Played - h.Home_Wins - a.Away_Wins - h.Home_Draws - a.Away_Draws) AS Total_Losses,
            SUM((h.Home_Wins + a.Away_Wins) * 100 / h.Games_Played) AS Win_Percentage
            
            FROM

            (SELECT HomeTeam, 
            COUNT (HomeTeam) * 2 AS Games_Played,
            SUM(FTHG) AS Home_Goals, 
            COUNT(CASE WHEN FTR = 'H' THEN 1 ELSE NULL END) AS Home_Wins,
            COUNT(CASE WHEN FTR = 'D' THEN 1 ELSE NULL END) AS Home_Draws
            FROM Matches
            WHERE Season = '2011'
            GROUP BY 1) h
            
            LEFT JOIN
            
            (SELECT AwayTeam,
            SUM(FTAG) AS Away_Goals,
            COUNT(CASE WHEN FTR = 'A' THEN 1 ELSE NULL END) AS Away_Wins,
            COUNT(CASE WHEN FTR = 'D' THEN 1 ELSE NULL END) AS Away_Draws
            FROM Matches
            WHERE Season = '2011'
            GROUP BY 1) a

            ON h.HomeTeam = a.AwayTeam
            GROUP BY 1
            ORDER BY Win_Percentage DESC
'''

combined_query = df_setup.read_sql(combined_query)
updated_df = combined_query
updated_df.head()

,Team,Games_Played,Total_Goals,Total_Wins,Total_Draws,Total_Losses,Win_Percentage
0,Man United,38,89,28,5,5,73
1,Man City,38,93,28,5,5,73
2,Dortmund,34,80,25,6,3,73
3,Bayern Munich,34,77,23,4,7,67
4,Schalke 04,34,74,20,4,10,58


In [62]:
data = {'Team Name':  ['First value', 'Second value'],
        'Total Goals 2011': ['First value', 'Second value'],
        'Total Wins 2011': ['First value', 'Second value'],
        'Rainy Win Percentage': ['First value', 'Second value']
       }
df = pd.DataFrame (data, columns = list(data.keys()))
df

,Team Name,Total Goals 2011,Total Wins 2011,Rainy Win Percentage
0,First value,First value,First value,First value
1,Second value,Second value,Second value,Second value
